In [35]:
import joblib
import torch
import librosa
import os
import cv2
import time
import numpy as np
        
lst = []
start_time = time.time()
path = "RAVDESS"

min_X = 1000000
min_frame = 1000

cnt = 0

for subdir, dirs, files in os.walk(path):
    for file in files:
        if file.endswith('.mp4'):
            X, sample_rate = librosa.load(os.path.join(subdir,file), res_type='kaiser_fast')
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
            video = []
            capture = cv2.VideoCapture(os.path.join(subdir,file))
            total_frame = capture.get(cv2.CAP_PROP_FRAME_COUNT)
            cnt = cnt + 1
            
            video = []
            frame_num = 16
            total_frame = capture.get(cv2.CAP_PROP_FRAME_COUNT) - 1
            df = (int)(total_frame / frame_num)

            while(capture.isOpened()):
                if capture.get(cv2.CAP_PROP_POS_FRAMES) + 1 == capture.get(cv2.CAP_PROP_FRAME_COUNT):
                    break
                elif (capture.get(cv2.CAP_PROP_POS_FRAMES) + 1) % df != 0 :
                    ret, frame = capture.read()
                    continue
                ret, frame = capture.read()
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                frame = cv2.resize(frame, (128, 128))
                video.append(frame)
                if len(video) == frame_num:
                    break
                    
            file = int(file[7:8]) - 1
            arr = video, X[:64916], file, mfccs
            lst.append(arr)
print("--- Video data loaded. Loading time : %s seconds ---" % (time.time() - start_time))

--- Video data loaded. Loading time : 1246.1424078941345 seconds ---


In [36]:
video_name = 'video.joblib'
audio_name = 'audio.joblib'
audio_mfcc_name = 'audio_mfcc.joblib'
y_name = 'y.joblib'

train_save_dir = 'RAVDESS_Model/train'
test_save_dir = 'RAVDESS_Model/test'

if not os.path.exists(train_save_dir):
        os.makedirs(train_save_dir)
if not os.path.exists(test_save_dir):
        os.makedirs(test_save_dir)

X1, X2, y, X3 = zip(*lst)
video = np.asarray(X1)
audio = np.asarray(X2)
audio_mfcc = np.asarray(X3)
y = np.asarray(y)

train_index = np.random.choice(2452, 2000, replace=False)
test_index = np.delete(np.asarray(range(2452)), index)

print(video.shape, audio.shape, audio_mfcc.shape, y.shape)

(2452, 16, 128, 128) (2452, 64916) (2452, 40) (2452,)


In [37]:
joblib.dump(video[train_index], os.path.join(train_save_dir, video_name))
joblib.dump(audio[train_index], os.path.join(train_save_dir, audio_name))
joblib.dump(audio_mfcc[train_index], os.path.join(train_save_dir, audio_mfcc_name))
joblib.dump(y[train_index], os.path.join(train_save_dir, y_name))

joblib.dump(video[test_index], os.path.join(test_save_dir, video_name))
joblib.dump(audio[test_index], os.path.join(test_save_dir, audio_name))
joblib.dump(audio_mfcc[test_index], os.path.join(test_save_dir, audio_mfcc_name))
joblib.dump(y[test_index], os.path.join(test_save_dir, y_name))

train_video = joblib.load(train_save_dir + '/video.joblib')
train_audio = joblib.load(train_save_dir + '/audio.joblib')
train_audio_mfcc = joblib.load(train_save_dir + '/audio_mfcc.joblib')
train_y = joblib.load(train_save_dir + '/y.joblib')

test_video = joblib.load(test_save_dir + '/video.joblib')
test_audio = joblib.load(test_save_dir + '/audio.joblib')
test_audio_mfcc = joblib.load(test_save_dir + '/audio_mfcc.joblib')
test_y = joblib.load(test_save_dir + '/y.joblib')

print(train_video.shape, train_audio.shape, train_audio_mfcc.shape, train_y.shape)
print(test_video.shape, test_audio.shape, test_audio_mfcc.shape, test_y.shape)

(2000, 16, 128, 128) (2000, 64916) (2000, 40) (2000,)
(452, 16, 128, 128) (452, 64916) (452, 40) (452,)
